In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install codecarbon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.9/494.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 45.0 MB/s eta 0:00:00


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from codecarbon import EmissionsTracker
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [11]:
# Caminho para o arquivo no Google Drive
caminho_arquivo = '/content/drive/MyDrive/Inteligência artificial/Mestrado/Aprendizado de Máquina/Trabalho/Experimentos/weatherAUS.csv'

# Ler o arquivo usando pandas
df = pd.read_csv(caminho_arquivo)

# Exibir as primeiras linhas do dataframe
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [12]:
df = df.drop(columns = 'Date')

df_cat = df.select_dtypes(include = ['object'])
for column_name in df_cat.columns:
    df.dropna(subset=[column_name], inplace=True)

numerical_cols = df.select_dtypes(include = ['float64', 'int64']).columns

# Cria o SimpleImputer com a estratégia desejada (por exemplo, preencher com a média)
imputer = SimpleImputer(strategy='mean')

# Aplica o imputer às colunas numéricas
df[numerical_cols] = imputer.fit_transform(df[numerical_cols])

replacement_dict = {'Yes': 1, 'No': 0}
df['RainTomorrow'] = df['RainTomorrow'].replace(replacement_dict)

X = df.drop('RainTomorrow', axis=1)
y = df['RainTomorrow']

categorical_cols = X.select_dtypes(include=['object']).columns

# Criar um ColumnTransformer para aplicar OneHotEncoder apenas nas colunas categóricas
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(), categorical_cols)],
    remainder='passthrough'
)

# Ajustar e transformar o DataFrame
X_encoded = ct.fit_transform(X)
y = df['RainTomorrow']

In [ ]:
'''X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


# Padronizar os dados
scaler = StandardScaler(with_mean=False)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


X_train = X_train.toarray()
X_test = X_test.toarray()'''

In [18]:
# Normalização dos dados
scaler = StandardScaler(with_mean=False)
X = scaler.fit_transform(X_encoded)

X = X.toarray()

In [22]:
tracker = EmissionsTracker()
tracker.start()


k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Função para criar o modelo
def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(32, kernel_initializer = 'uniform',activation='relu'))
    model.add(Dense(16, kernel_initializer = 'uniform', activation='relu'))
    model.add(Dense(8, kernel_initializer = 'uniform',activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Armazenar as métricas de desempenho
accuracies = []
precisions = []
recalls = []

# Realizar o K-Fold Cross Validation
for train_index, test_index in kf.split(X):
    X_train, X_val = X[train_index], X[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]

    model = create_model()
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

    # Fazer previsões
    y_pred = model.predict(X_val)
    y_pred = (y_pred > 0.5).astype(int).flatten()  # Convertendo probabilidades para classes binárias

    # Calcular métricas
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)

# Calcular a média e desvio padrão das métricas
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)

mean_precision = np.mean(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
std_recall = np.std(recalls)

print(f"Acurácia Média: {mean_accuracy:.4f}")
print(f"Desvio Padrão da Acurácia: {std_accuracy:.4f}")

print(f"Precisão Média: {mean_precision:.4f}")
print(f"Desvio Padrão da Precisão: {std_precision:.4f}")

print(f"Recall Médio: {mean_recall:.4f}")
print(f"Desvio Padrão do Recall: {std_recall:.4f}")

tracker.stop()
tracker.final_emissions_data

[codecarbon WARNING @ 22:12:53] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 22:12:53] [setup] RAM Tracking...
[codecarbon INFO @ 22:12:53] [setup] GPU Tracking...
[codecarbon INFO @ 22:12:53] No GPU found.
[codecarbon INFO @ 22:12:53] [setup] CPU Tracking...
[codecarbon WARNING @ 22:12:53] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 22:12:55] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 22:12:55] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 22:12:55] >>> Tracker's metadata:
[codecarbon INFO @ 22:12:55]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 22:12:55]   Python version: 3.10.12
[codecarbon INFO @ 22:12:55]   CodeCarbon version: 2.4.2
[codecarbon INFO @ 22:12:55]   Available RAM : 12.675 GB
[codecarbon INFO @ 22:12:55]   CPU count: 2
[codecarbon INFO @ 22:12:55]   CP

Epoch 1/100
2934/3093 [===========================>..] - ETA: 0s - loss: 0.3964 - accuracy: 0.8278

[codecarbon INFO @ 22:13:05] Energy consumed for RAM : 0.000079 kWh. RAM Power : 4.753040313720703 W
[codecarbon INFO @ 22:13:05] Energy consumed for all CPUs : 0.000708 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:13:05] 0.000787 kWh of electricity used since the beginning.


3093/3093 [==============================] - 10s 2ms/step - loss: 0.3947 - accuracy: 0.8287
Epoch 2/100
1875/3093 [=================>............] - ETA: 2s - loss: 0.3586 - accuracy: 0.8458

[codecarbon INFO @ 22:13:10] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.753040313720703 W
[codecarbon INFO @ 22:13:10] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:13:10] 0.000197 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 23:32:11] Energy consumed for RAM : 0.006274 kWh. RAM Power : 4.753040313720703 W
[codecarbon INFO @ 23:32:11] Energy consumed for all CPUs : 0.056122 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:32:11] 0.062396 kWh of electricity used since the beginning.


774/774 [==============================] - 1s 2ms/step


[codecarbon INFO @ 23:32:21] Energy consumed for RAM : 0.006287 kWh. RAM Power : 4.753040313720703 W
[codecarbon INFO @ 23:32:21] Energy consumed for all CPUs : 0.056238 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:32:21] 0.062525 kWh of electricity used since the beginning.


Acurácia Média: 0.8585
Desvio Padrão da Acurácia: 0.0024
Precisão Média: 0.7340
Desvio Padrão da Precisão: 0.0291
Recall Médio: 0.5713
Desvio Padrão do Recall: 0.0425


EmissionsData(timestamp='2024-06-10T23:32:21', project_name='codecarbon', run_id='37e81ad7-1f44-4ccf-aa2d-24ca549f5697', duration=4766.181548833847, emissions=0.01785118368143744, emissions_rate=3.7453847484691664e-06, cpu_power=42.5, gpu_power=0.0, ram_power=4.753040313720703, cpu_energy=0.05623786814560499, gpu_energy=0, ram_energy=0.006287350578180224, energy_consumed=0.06252521872378523, country_name='United States', country_iso_code='USA', region='south carolina', cloud_provider='', cloud_region='', os='Linux-6.1.85+-x86_64-with-glibc2.35', python_version='3.10.12', codecarbon_version='2.4.2', cpu_count=2, cpu_model='Intel(R) Xeon(R) CPU @ 2.20GHz', gpu_count=None, gpu_model=None, longitude=-79.9746, latitude=32.8608, ram_total_size=12.674774169921875, tracking_mode='machine', on_cloud='N', pue=1.0)

In [ ]:
model = Sequential()

# Adicionar camadas densamente conectadas (fully connected layers)
model.add(Dense(units=16, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=4, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))  # Camada de saída para classificação binária

# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
tracker = EmissionsTracker()
tracker.start()

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

emissao_RNA = tracker.stop()

[codecarbon WARNING @ 22:00:35] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 22:00:35] [setup] RAM Tracking...
[codecarbon INFO @ 22:00:35] [setup] GPU Tracking...
[codecarbon INFO @ 22:00:35] No GPU found.
[codecarbon INFO @ 22:00:35] [setup] CPU Tracking...
[codecarbon WARNING @ 22:00:35] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 22:00:36] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 22:00:36] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 22:00:36] >>> Tracker's metadata:
[codecarbon INFO @ 22:00:36]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 22:00:36]   Python version: 3.10.12
[codecarbon INFO @ 22:00:36]   CodeCarbon version: 2.4.2
[codecarbon INFO @ 22:00:36]   Available RAM : 12.675 GB
[codecarbon INFO @ 22:00:36]   CPU count: 2
[codecarbon INFO @ 22:00:36]   CP

Epoch 1/50
2475/2475 [==============================] - 6s 2ms/step - loss: 0.3538 - accuracy: 0.8477 - val_loss: 0.3542 - val_accuracy: 0.8492
Epoch 2/50
2475/2475 [==============================] - 8s 3ms/step - loss: 0.3510 - accuracy: 0.8489 - val_loss: 0.3554 - val_accuracy: 0.8487
Epoch 3/50
 605/2475 [======>.......................] - ETA: 3s - loss: 0.3450 - accuracy: 0.8506

[codecarbon INFO @ 22:00:52] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.753040313720703 W
[codecarbon INFO @ 22:00:52] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:00:52] 0.000197 kWh of electricity used since the beginning.


2475/2475 [==============================] - 5s 2ms/step - loss: 0.3475 - accuracy: 0.8507 - val_loss: 0.3633 - val_accuracy: 0.8434
Epoch 4/50
2475/2475 [==============================] - 6s 3ms/step - loss: 0.3457 - accuracy: 0.8513 - val_loss: 0.3488 - val_accuracy: 0.8522
Epoch 5/50
1617/2475 [==================>...........] - ETA: 2s - loss: 0.3426 - accuracy: 0.8543

[codecarbon INFO @ 22:01:07] Energy consumed for RAM : 0.000040 kWh. RAM Power : 4.753040313720703 W


...

[codecarbon INFO @ 22:05:52] Energy consumed for RAM : 0.000416 kWh. RAM Power : 4.753040313720703 W
[codecarbon INFO @ 22:05:52] Energy consumed for all CPUs : 0.003718 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:05:52] 0.004134 kWh of electricity used since the beginning.


2475/2475 [==============================] - 8s 3ms/step - loss: 0.3229 - accuracy: 0.8612 - val_loss: 0.3359 - val_accuracy: 0.8539
Epoch 49/50
2475/2475 [==============================] - 6s 2ms/step - loss: 0.3213 - accuracy: 0.8611 - val_loss: 0.3345 - val_accuracy: 0.8570
Epoch 50/50
2180/2475 [=========================>....] - ETA: 0s - loss: 0.3226 - accuracy: 0.8610

[codecarbon INFO @ 22:06:07] Energy consumed for RAM : 0.000435 kWh. RAM Power : 4.753040313720703 W
[codecarbon INFO @ 22:06:07] Energy consumed for all CPUs : 0.003895 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:06:07] 0.004331 kWh of electricity used since the beginning.


2475/2475 [==============================] - 7s 3ms/step - loss: 0.3221 - accuracy: 0.8614 - val_loss: 0.3409 - val_accuracy: 0.8528


[codecarbon INFO @ 22:06:22] Energy consumed for RAM : 0.000455 kWh. RAM Power : 4.753040313720703 W
[codecarbon INFO @ 22:06:22] Energy consumed for all CPUs : 0.004072 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:06:22] 0.004528 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:06:37] Energy consumed for RAM : 0.000475 kWh. RAM Power : 4.753040313720703 W
[codecarbon INFO @ 22:06:37] Energy consumed for all CPUs : 0.004249 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:06:37] 0.004724 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:06:52] Energy consumed for RAM : 0.000495 kWh. RAM Power : 4.753040313720703 W
[codecarbon INFO @ 22:06:52] Energy consumed for all CPUs : 0.004426 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:06:52] 0.004921 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:06:59] Energy consumed for RAM : 0.000504 kWh. RAM Power : 4.753040313720703 W
[codecarbon INFO @ 22:06:59] Energy consumed for a